# AIF360 - total_credit_scoring

Here the following notebook is replicated.
There are some adaptations in order to get (easy) access to the data.

ref. https://github.com/Trusted-AI/AIF360/blob/main/examples/tutorial_credit_scoring.ipynb

In [1]:
# Install the aif360 package

!pip install -qU aif360
!pip install -qU 'aif360[Reductions]'
!pip install -qU 'aif360[inFairness]'

In [11]:
# Load all necessary packages

import numpy as np
import pandas as pd
np.random.seed(0)

# Code from the original dataset, it is replaced by the code in the cell below.
# This: from aif360.datasets import GermanDataset -> replace by using the class "MyGermanDataset" to which
# you add the path where you've stored the "german.data" file.

from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

import warnings                   # to deal with warnings
warnings.filterwarnings('ignore')

In [ ]:
# If you're working with Google Colab and the data is located in your Google Drive:

# Mount your google drive.
from google.colab import drive
drive.mount('/content/drive')

# The path is something like: "/content/drive/MyDrive/Colab Data/german.data", for example.

In [12]:
# Instead of using the GermanDataset class, which loads the "german.data" file from a location relative to the file you're running,
# you can save the "german.data" file in a location of your choice and load it from there.

# Therefore, use the class "MyGermanDataset" defined in this cell, passing it the "path" of the file, instead of the predefined
# Germandataset from aif360.datasets.

# Download the "german.data" file from:
# https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data

default_mappings = {
    'label_maps': [{1.0: 'Good Credit', 2.0: 'Bad Credit'}],
    'protected_attribute_maps': [{1.0: 'Male', 0.0: 'Female'},
                                 {1.0: 'Old', 0.0: 'Young'}],
}
def default_preprocessing(df):
    """Adds a derived sex attribute based on personal_status."""
    # TODO: ignores the value of privileged_classes for 'sex'
    status_map = {'A91': 'male', 'A93': 'male', 'A94': 'male',
                  'A92': 'female', 'A95': 'female'}
    df['sex'] = df['personal_status'].replace(status_map)

    return df

class MyGermanDataset(StandardDataset):
    """German credit Dataset.

    See :file:`aif360/data/raw/german/README.md`.
    """

    def __init__(self, label_name='credit', favorable_classes=[1],
                 protected_attribute_names=['sex', 'age'],
                 privileged_classes=[['male'], lambda x: x > 25],
                 instance_weights_name=None,
                 categorical_features=['status', 'credit_history', 'purpose',
                     'savings', 'employment', 'other_debtors', 'property',
                     'installment_plans', 'housing', 'skill_level', 'telephone',
                     'foreign_worker'],
                 features_to_keep=[], features_to_drop=['personal_status'],
                 na_values=[], custom_preprocessing=default_preprocessing,
                 metadata=default_mappings,
                 path = None):
        """See :obj:`StandardDataset` for a description of the arguments.

        By default, this code converts the 'age' attribute to a binary value
        where privileged is `age > 25` and unprivileged is `age <= 25` as
        proposed by Kamiran and Calders [1]_.

        References:
            .. [1] F. Kamiran and T. Calders, "Classifying without
               discriminating," 2nd International Conference on Computer,
               Control and Communication, 2009.

        Examples:
            In some cases, it may be useful to keep track of a mapping from
            `float -> str` for protected attributes and/or labels. If our use
            case differs from the default, we can modify the mapping stored in
            `metadata`:

            >>> label_map = {1.0: 'Good Credit', 0.0: 'Bad Credit'}
            >>> protected_attribute_maps = [{1.0: 'Male', 0.0: 'Female'}]
            >>> gd = GermanDataset(protected_attribute_names=['sex'],
            ... privileged_classes=[['male']], metadata={'label_map': label_map,
            ... 'protected_attribute_maps': protected_attribute_maps})

            Now this information will stay attached to the dataset and can be
            used for more descriptive visualizations.
        """

        #filepath = os.path.join(os.path.dirname(os.path.abspath(__file__)),
        #                        '..', 'data', 'raw', 'german', 'german.data')
        filepath = path
        # as given by german.doc
        column_names = ['status', 'month', 'credit_history',
            'purpose', 'credit_amount', 'savings', 'employment',
            'investment_as_income_percentage', 'personal_status',
            'other_debtors', 'residence_since', 'property', 'age',
            'installment_plans', 'housing', 'number_of_credits',
            'skill_level', 'people_liable_for', 'telephone',
            'foreign_worker', 'credit']
        try:
            df = pd.read_csv(filepath, sep=' ', header=None, names=column_names,
                             na_values=na_values)
        except IOError as err:
            print("IOError: {}".format(err))
            print("To use this class, please download the following files:")
            print("\n\thttps://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data")
            print("\thttps://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc")
            print("\nand place them, as-is, in the folder:")
            print("\n\t{}\n".format(os.path.abspath(os.path.join(
                os.path.abspath(__file__), '..', '..', 'data', 'raw', 'german'))))
            import sys
            sys.exit(1)

        super(MyGermanDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)

## Step 2 Load dataset, specifying protected attribute, and split dataset into train and test

In Step 2 we load the initial dataset, setting the protected attribute to be age. We then split the original dataset into training and testing datasets. Although we will use only the training dataset in this tutorial, a normal workflow would also use a test dataset for assessing the efficacy (accuracy, fairness, etc.) during the development of a machine learning model. 

Finally, we set two variables (to be used in Step 3) for the privileged (1) and unprivileged (0) values for the age attribute. These are key inputs for detecting and mitigating bias, which will be Step 3 and Step 4.

In [17]:
dataset_orig = MyGermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'], # ignore sex-related attributes
    path = "replace_this_by_your_(relative)_file_path"      # I've put the file in "./data/german.data"
)


print(type(dataset_orig))
print(dir(dataset_orig))

a = dataset_orig.convert_to_dataframe()
print(a[0].head())

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

<class '__main__.MyGermanDataset'>
['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_de_dummy_code_df', '_parse_feature_names', 'align_datasets', 'convert_to_dataframe', 'copy', 'export_dataset', 'favorable_label', 'feature_names', 'features', 'ignore_fields', 'import_dataset', 'instance_names', 'instance_weights', 'label_names', 'labels', 'metadata', 'privileged_protected_attributes', 'protected_attribute_names', 'protected_attributes', 'scores', 'split', 'subset', 'temporarily_ignore', 'unfavorable_label', 'unprivileged_protected_attributes', 'validate_dataset']
   month  credit_amount  investment_as_income_percentage  residence

In [34]:
print(set(dataset_orig.instance_weights))
for el in set(dataset_orig.instance_weights):
    print(el)

{1.0}
1.0


## Step 3 Compute fairness metric on original training dataset

Now that we've identified the protected attribute 'age' and defined privileged and unprivileged values, we can use aif360 to detect **bias in the dataset**. 

One simple test is to compare the percentage of favorable results for the privileged and unprivileged groups, subtracting the former percentage from the latter. 

A negative value indicates less favorable outcomes for the unprivileged groups. 

This is implemented in the method called mean_difference on the BinaryLabelDatasetMetric class. 

The code below performs this check and displays the output, showing that the difference is -0.169905.

In [27]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.129916


## Step 4 Mitigate bias by transforming the original dataset

The previous step showed that the privileged group was getting 13% more positive outcomes in the training dataset. 

Since this is not desirable, we are going to try to mitigate this bias in the training dataset. As stated above, this is called **pre-processing mitigation** because it happens before the creation of the model.

AI Fairness 360 implements several pre-processing mitigation algorithms. We will choose the **Reweighing algorithm**, which is implemented in the Reweighing class in the aif360.algorithms.preprocessing package. This algorithm will transform the dataset to have more equity in positive outcomes on the protected attribute for the privileged and unprivileged groups.

We then call the fit and transform methods to perform the transformation, producing a newly transformed training dataset (dataset_transf_train).

In [28]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

## Step 5 Compute fairness metric on transformed dataset

Now that we have a transformed dataset, we can check how effective it was in removing bias by using the same metric we used for the original training dataset in Step 3. 

Once again, we use the function mean_difference in the BinaryLabelDatasetMetric class. 

We see the mitigation step was very effective, the difference in mean outcomes is now 0.0. So we went from a 13% advantage for the privileged group to equality in terms of mean outcome.

In [29]:
metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000
